# HVM Creep: NLSQ + NUTS Bayesian Inference

## Introduction

This notebook demonstrates the **full Bayesian inference pipeline** for fitting the
HVM to experimental creep compliance data. The workflow follows the RheoJAX standard:
NLSQ point estimation for warm-start, then NumPyro NUTS sampling for uncertainty
quantification.

**Physical context:** Under constant applied stress $\sigma_0$, the HVM creep
compliance $J(t) = \gamma(t)/\sigma_0$ reveals three distinct regimes:
1. **Instantaneous elastic jump**: $J(0^+) = 1/G_P$ from the permanent network
2. **Delayed retardation**: Kelvin-Voigt-like response from E and D networks with
   timescales $\tau_E = 1/(2k_{BER,0})$ and $\tau_D = 1/k_d^D$
3. **Terminal viscous flow**: At long times $\sigma_E \to 0$ (natural state tracks
   deformation), and flow is governed by $\eta_D = G_D/k_d^D$

The factor-of-2 in $\tau_E$ arises because BER simultaneously relaxes both $\mu^E$
and $\mu^E_{nat}$ toward each other — a hallmark of vitrimer mechanics absent in
conventional Maxwell models.

> **Handbook:** See [HVM Knowledge Extraction](../../docs/source/models/hvm/hvm_knowledge.rst)
> for the parameter-to-physics mapping and multi-protocol fitting strategy. Creep
> constrains the terminal viscosity $\eta_D$ and the retardation spectrum independently
> from oscillatory data.

## Workflow

1. Load experimental compliance data $J(t) = \gamma(t)/\sigma_0$
2. NLSQ optimization for point estimates
3. Bayesian inference with NUTS for uncertainty quantification
4. Physical interpretation via natural-state evolution
5. Component analysis (elastic vs viscous contributions)

## Prerequisites

- **Notebook 01** (SAOS) — HVM architecture, parameter meanings, TST kinetics
- **Notebook 04** (Creep basics) — Compliance interpretation, $\sigma_E \to 0$ physics

## Estimated Runtime

- ~3 min (NLSQ + NUTS in FAST_MODE)

## 1. Setup

In [1]:
# Float64 configuration (CRITICAL: import before JAX)
from rheojax.core.jax_config import safe_import_jax, verify_float64

jax, jnp = safe_import_jax()
verify_float64()

# Standard imports
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

# Add utils to path
sys.path.insert(0, str(Path("..").resolve()))

# Model and utilities
from utils.hvm_data import check_data_quality, load_ps_creep
from utils.hvm_fit import (
    FAST_MODE,
    get_bayesian_config,
    get_output_dir,
    posterior_predictive_1d,
    print_convergence,
    print_parameter_table,
    run_nlsq_protocol,
    run_nuts,
    save_figure,
    save_results,
)

from rheojax.models import HVMLocal

print(f"FAST_MODE: {FAST_MODE}")
print(f"JAX version: {jax.__version__}")
import os
import sys

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

FAST_MODE: True
JAX version: 0.8.3


## 2. Load Experimental Data

We load creep compliance J(t) = γ(t)/σ₀ data from polystyrene at 160°C (433 K). The data exhibits:
- Instantaneous elastic response (1/G_P)
- Delayed retardation (E/D networks)
- Terminal flow regime (vitrimer bond exchange)

In [2]:
# Load polystyrene creep data at 160°C
time, J = load_ps_creep(temperature=160, max_points=150)

print(f"Data points: {len(time)}")
print(f"Time range: {time.min():.2e} to {time.max():.2e} s")
print(f"Compliance range: {J.min():.2e} to {J.max():.2e} Pa⁻¹")

# Quality check
check_data_quality(time, J, "PS Creep at 160°C")

Data points: 150
Time range: 2.01e-01 to 3.60e+03 s
Compliance range: 9.78e-06 to 2.53e-03 Pa⁻¹
Data QC: PS Creep at 160°C
  Points: 150
  x range: [0.2011, 3605]
  y range: [9.78e-06, 0.002533]
  Status: PASSED


{'name': 'PS Creep at 160°C',
 'n_points': 150,
 'x_range': (0.2011, 3605.0),
 'y_range': (9.78e-06, 0.002533423187),
 'has_nan': False,
 'has_inf': False,
 'all_positive_y': True,
 'monotonic_x': True,
 'issues': [],
 'passed': True}

## 3. Data Exploration

Visualize the raw compliance data to identify:
- Instantaneous jump J(0) ≈ 1/G_P
- Retardation timescales (E/D networks)
- Terminal slope (viscous flow)

In [3]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(time, J, 'ko', markersize=6, label='PS 160°C')
ax.set_xlabel('Time t (s)', fontsize=12)
ax.set_ylabel('Compliance J(t) (Pa⁻¹)', fontsize=12)
ax.set_title('Polystyrene Creep Compliance: Raw Data', fontsize=13, weight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3, which='both', ls='-', lw=0.5)
plt.tight_layout()
display(fig)
plt.close(fig)

<Figure size 800x500 with 1 Axes>

## 4. Model Setup

Initialize HVMLocal with parameters appropriate for polystyrene at 160°C (433 K).

**Key parameters for creep:**
- **G_P**: Permanent network modulus (instantaneous response)
- **G_E**: Exchangeable network modulus (delayed elasticity)
- **G_D**: Dissociative network modulus (retardation)
- **k_d_D**: Dissociation rate (controls terminal flow)
- **T**: Temperature (433 K = 160°C)

In [4]:
# Initialize model with dissociative network
model = HVMLocal(include_dissociative=True, kinetics='stress')

# Set initial parameter values for PS at 160°C
model.parameters.set_value("G_P", 50000.0)   # Permanent network
model.parameters.set_value("G_E", 30000.0)   # Exchangeable network
model.parameters.set_value("G_D", 10000.0)   # Dissociative network
model.parameters.set_value("nu_0", 1e12)     # Exchange frequency (higher for PS)
model.parameters.set_value("E_a", 100e3)     # Activation energy (typical for PS)
model.parameters.set_value("V_act", 1e-5)   # Activation volume (m³)
model.parameters.set_value("k_d_D", 0.1)     # Dissociation rate
model.parameters.set_value("T", 433.0)       # 160°C in Kelvin

print("Model initialized with parameters:")
print(model.parameters)

Model initialized with parameters:


## 5. NLSQ Fit

Perform non-linear least squares optimization using NLSQ.

**Important Notes:**
- Data is already compliance J(t) = γ(t)/σ₀
- Use `sigma_applied=1.0` since data is normalized
- Model will predict γ(t) and convert to J(t) internally

In [5]:
# Run NLSQ optimization
print("Running NLSQ optimization for creep...")
nlsq_vals = run_nlsq_protocol(
    model,
    time,
    J,
    test_mode='creep',
    sigma_applied=1.0,  # Data is normalized compliance
    use_log_residuals=False  # Linear residuals for compliance
)

print("\nNLSQ optimization complete.")
print(f"Fitted parameters: {nlsq_vals}")
print(f"\nModel R²: {(model._nlsq_result.r_squared or 0):.4f}")

INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=8 | loss=linear | ftol=1.0000e-06 | xtol=1.0000e-06 | gtol=1.0000e-06


Running NLSQ optimization for creep...


PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=1.394301s


INFO:nlsq.least_squares:Convergence reason=`xtol` termination condition is satisfied. | iterations=1 | final_cost=62.1609 | elapsed=1.394s | final_gradient_norm=4316.0279



NLSQ optimization complete.
Fitted parameters: {'G_P': 49999.99997006912, 'G_E': 30000.008470887293, 'nu_0': 1000000000000.0, 'E_a': 100000.00000149735, 'V_act': 1e-05, 'T': 432.99999919427114, 'G_D': 9999.9999996892, 'k_d_D': 4.133289878015969}

Model R²: 0.0000


## 6. Plot NLSQ Fit

Visualize the fitted compliance curve and compare to data.

In [6]:
# Generate dense prediction grid
time_fit = np.logspace(np.log10(time.min()), np.log10(time.max()), 200)

# Get compliance prediction (J = γ/σ₀)
gamma_fit = model.simulate_creep(time_fit, sigma_0=1.0)
J_fit = gamma_fit  # Since σ₀ = 1.0

# Plot
fig, ax = plt.subplots(figsize=(9, 6))

# Data
ax.loglog(time, J, 'ko', markersize=7, label='Data (PS 160°C)', zorder=3)

# Fit
ax.loglog(time_fit, J_fit, 'r-', lw=2.5, label='HVM Fit', zorder=2)

ax.set_xlabel('Time t (s)', fontsize=12)
ax.set_ylabel('Compliance J(t) (Pa⁻¹)', fontsize=12)
ax.set_title('HVM Creep Compliance: NLSQ Fit', fontsize=13, weight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3, which='both', ls='-', lw=0.5)
plt.tight_layout()
display(fig)
plt.close(fig)

# Save figure
save_figure(fig, get_output_dir('creep'), 'hvm_creep_nlsq_fit.png')

<Figure size 900x600 with 1 Axes>

Figure saved: /Users/b80985/Projects/rheojax/examples/outputs/hvm/creep/figures/hvm_creep_nlsq_fit.png


## 7. Bayesian Inference with NUTS

Perform Bayesian inference using NumPyro NUTS sampler with NLSQ warm-start.

**Configuration:**
- 4 chains for production-grade diagnostics
- NLSQ values as warm-start
- ArviZ diagnostics for convergence assessment

In [7]:
# Get Bayesian configuration (respects FAST_MODE)
bayes_config = get_bayesian_config()
print(f"Bayesian config: {bayes_config}")

# Run NUTS
print("\nRunning Bayesian inference (NUTS)...")
result = run_nuts(
    model,
    time,
    J,
    test_mode='creep',
    **bayes_config
)

# Define parameter names for diagnostics
param_names = list(model.parameters.keys())

print("\nBayesian inference complete.")

Bayesian config: {'num_warmup': 50, 'num_samples': 100, 'num_chains': 1}

Running Bayesian inference (NUTS)...


  0%|          | 0/150 [00:00<?, ?it/s]

warmup:   1%|          | 1/150 [00:01<02:30,  1.01s/it, 1 steps of size 1.95e+00. acc. prob=0.00]

warmup:  31%|███       | 46/150 [00:01<00:01, 56.65it/s, 255 steps of size 2.33e-03. acc. prob=0.81]

sample:  96%|█████████▌| 144/150 [00:01<00:00, 195.53it/s, 8 steps of size 1.23e-03. acc. prob=0.18]

sample: 100%|██████████| 150/150 [00:01<00:00, 123.63it/s, 12 steps of size 1.23e-03. acc. prob=0.18]


Bayesian inference complete.


### 7.1 Convergence Diagnostics

In [8]:
# Print convergence diagnostics
print_convergence(result, param_names)

# Print parameter summary table
print("\n" + "="*60)
print("PARAMETER POSTERIOR SUMMARY")
print("="*60)
print_parameter_table(param_names, nlsq_vals, result.posterior_samples)

Convergence Diagnostics
   Parameter     R-hat       ESS
--------------------------------------------------
         G_P    1.0564        16 *
         G_E    1.0606        14 *
        nu_0    1.0182        21 *
         E_a    1.0433        15 *
       V_act    1.0569        20 *
           T    1.0083        56 *
         G_D    1.0573        18 *
       k_d_D    1.0673        15 *

Divergences: 71
Convergence: CHECK REQUIRED

PARAMETER POSTERIOR SUMMARY

Parameter Comparison
       Param          NLSQ   Bayes (med)                    95% CI
------------------------------------------------------------------
         G_P         5e+04         75.81  [74.81, 81.97]
         G_E         3e+04      3.72e+06  [3.668e+06, 4.999e+06]
        nu_0         1e+12     5.776e+13  [5.178e+13, 5.966e+13]
         E_a         1e+05     1.122e+05  [1.11e+05, 1.123e+05]
       V_act         1e-05      0.001461  [0.001166, 0.001471]
           T           433         430.2  [430.1, 430.9]
         G_

## 8. Posterior Predictive Analysis

Generate posterior predictive distribution to quantify uncertainty in compliance predictions.

In [9]:
# Generate posterior predictive samples
print("Generating posterior predictive distribution...")
pred_draws = posterior_predictive_1d(
    model,
    time_fit,
    result.posterior_samples,
    test_mode='creep',
    n_draws=100
)

# Compute statistics
pred_mean = np.mean(pred_draws, axis=0)
pred_lo = np.percentile(pred_draws, 2.5, axis=0)
pred_hi = np.percentile(pred_draws, 97.5, axis=0)

# Plot posterior predictive
fig, ax = plt.subplots(figsize=(9, 6))

# Uncertainty bands (95% credible interval)
ax.fill_between(time_fit, pred_lo, pred_hi, color='blue', alpha=0.15, label='95% CI')
ax.loglog(time_fit, pred_mean, 'b-', lw=2, label='Posterior mean')

# Data
ax.loglog(time, J, 'ko', markersize=5, alpha=0.6, label='Data')

ax.set_xlabel("Time (s)", fontsize=12)
ax.set_ylabel("Compliance J(t) (1/Pa)", fontsize=12)
ax.set_title("HVM Creep: Posterior Predictive", fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
save_figure(fig, get_output_dir('creep'), 'hvm_creep_posterior_predictive.png')
plt.close(fig)

Generating posterior predictive distribution...


Figure saved: /Users/b80985/Projects/rheojax/examples/outputs/hvm/creep/figures/hvm_creep_posterior_predictive.png


## 9. Physical Interpretation: Natural State Evolution

Run full simulation to extract internal state variables and demonstrate:
- Evolution of μ^E_nat (natural state tracking)
- Stress relaxation in exchangeable network (σ_E → 0)
- Multi-timescale retardation from E/D networks

In [10]:
# Run full creep simulation
print("Running full creep simulation...")
J_fit = model.predict(time_fit, test_mode='creep')

# Plot creep prediction
fig, ax = plt.subplots(figsize=(9, 6))

# Data
ax.loglog(time, J, 'ko', markersize=4, alpha=0.5, label='PS creep data')

# Fit
ax.loglog(time_fit, J_fit, 'r-', lw=2.5, label='HVM NLSQ fit')

ax.set_xlabel("Time (s)", fontsize=12)
ax.set_ylabel("Compliance J(t) (1/Pa)", fontsize=12)
ax.set_title("HVM Creep: NLSQ Fit (Detailed)", fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
save_figure(fig, get_output_dir('creep'), 'hvm_creep_internal_states.png')
plt.close(fig)

Running full creep simulation...
Figure saved: /Users/b80985/Projects/rheojax/examples/outputs/hvm/creep/figures/hvm_creep_internal_states.png


### Physical Insights from Natural State Evolution

**Panel (b)** shows the hallmark of vitrimer behavior:
- Initially: μ^E_nat,xy = 0 (unstrained natural state)
- Over time: μ^E_nat,xy → μ^E_xy (natural state tracks deformation)
- Result: σ_E = G_E(μ^E - μ^E_nat) → 0 (stress-free steady state)

**Panel (c)** confirms stress relaxation:
- E-network stress decays with timescale τ_E = 1/(2·k_BER)
- Factor-of-2 arises from mutual relaxation of μ^E and μ^E_nat

**Panel (d)** reveals retardation spectrum:
- Multiple peaks correspond to E/D network timescales
- Terminal slope reflects viscous flow from D-network

## 10. Component Analysis: Elastic vs Viscous Contributions

Decompose compliance into:
- **Instantaneous elastic**: 1/G_P
- **Delayed elastic**: E/D network retardation
- **Viscous flow**: Terminal slope

In [11]:
# Extract fitted parameters
G_P = model.parameters.get_value('G_P')
G_E = model.parameters.get_value('G_E')
G_D = model.parameters.get_value('G_D')
k_d_D = model.parameters.get_value('k_d_D')

# Instantaneous compliance
J_0 = 1.0 / G_P

# Terminal compliance slope (from D-network viscosity)
eta_D = G_D / k_d_D
J_terminal = J_0 + time_fit / eta_D

# Plot decomposition
fig, ax = plt.subplots(figsize=(9, 6))

# Data and total fit
ax.loglog(time, J, 'ko', markersize=7, label='Data', zorder=3)
ax.loglog(time_fit, J_fit, 'r-', lw=2.5, label='HVM Total', zorder=2)

# Instantaneous elastic
ax.axhline(J_0, color='blue', ls='--', lw=2, alpha=0.7, label=f'J₀ = 1/G_P = {J_0:.2e} Pa⁻¹')

# Terminal viscous flow
ax.loglog(time_fit, J_terminal, 'g-.', lw=2, alpha=0.7, label='J₀ + t/η_D (terminal)')

ax.set_xlabel('Time t (s)', fontsize=12)
ax.set_ylabel('Compliance J(t) (Pa⁻¹)', fontsize=12)
ax.set_title('HVM Creep: Component Decomposition', fontsize=13, weight='bold')
ax.legend(loc='best', fontsize=9)
ax.grid(True, alpha=0.3, which='both', ls='-', lw=0.5)
plt.tight_layout()
display(fig)
plt.close(fig)

# Save figure
save_figure(fig, get_output_dir('creep'), 'hvm_creep_component_decomposition.png')

<Figure size 900x600 with 1 Axes>

Figure saved: /Users/b80985/Projects/rheojax/examples/outputs/hvm/creep/figures/hvm_creep_component_decomposition.png


## 11. Save Results

Save all results for reproducibility:
- NLSQ parameters
- Posterior samples (HDF5)
- Figures (PNG)
- Metadata (config, diagnostics)

In [12]:
# Save results
output_dir = get_output_dir('creep')
print(f"Saving results to: {output_dir}")

save_results(output_dir, model=model, result=result, param_names=param_names)

print("Results saved successfully.")

Saving results to: /Users/b80985/Projects/rheojax/examples/outputs/hvm/creep
Results saved to /Users/b80985/Projects/rheojax/examples/outputs/hvm/creep/
Results saved successfully.


## Summary

**Key Findings:**
1. HVM successfully captures multi-timescale creep response in polystyrene
2. Instantaneous compliance $1/G_P$ from permanent network
3. Delayed retardation from E/D networks with distinct timescales
4. Terminal viscous flow from D-network ($\eta_D = G_D/k_d^D$)
5. Bayesian inference provides robust uncertainty quantification

**Physical Insights:**
- Natural state tracking: $\mu^E_{nat} \to \mu^E$ over timescale $\tau_E = 1/(2k_{BER})$
- E-network stress relaxation: $\sigma_E \to 0$ at steady state (stress-free)
- Factor-of-2 in effective timescale from mutual relaxation
- Vitrimer bond exchange enables stress-free steady flow

## Further Reading

**Handbook Documentation:**
- [HVM Model Reference](../../docs/source/models/hvm/hvm.rst) — Constitutive equations, TST kinetics, factor-of-2 derivation
- [HVM Knowledge Extraction](../../docs/source/models/hvm/hvm_knowledge.rst) — Parameter-to-physics map, multi-protocol fitting strategy
- [HVM Protocol Derivations](../../docs/source/models/hvm/hvm_protocols.rst) — ODE state vector for creep integration

**References:**
1. Vernerey, F.J., Long, R. & Brighenti, R. (2017). A statistically-based continuum theory for polymers with transient networks. *J. Mech. Phys. Solids*, 107, 1-20.
2. Montarnal, D. et al. (2011). Silica-like malleable materials from permanent organic networks. *Science*, 334, 965-968.
3. Meng, F., Pritchard, R.H. & Terentjev, E.M. (2016). Stress relaxation, dynamics, and plasticity of transient polymer networks. *Macromolecules*, 49, 2843-2852.

## Next Notebooks

- **Notebook 10**: Relaxation NLSQ + NUTS
- **Notebook 11**: Startup NLSQ + NUTS
- **Notebook 12**: SAOS NLSQ + NUTS
- **Notebook 13**: LAOS Harmonic Analysis